<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/all_seeds_combine_Adam_single%2Bneurons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch.utils.data import DataLoader, Subset
import torchvision
import torchvision.transforms as transforms
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

root_dir = './'
torchvision.datasets.MNIST(root=root_dir,download=True)

--2021-04-09 07:29:38--  http://www.di.ens.fr/~lelarge/MNIST.tar.gz
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.di.ens.fr/~lelarge/MNIST.tar.gz [following]
--2021-04-09 07:29:38--  https://www.di.ens.fr/~lelarge/MNIST.tar.gz
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘MNIST.tar.gz’

MNIST.tar.gz            [       <=>          ]  33.20M  2.87MB/s    in 20s     

2021-04-09 07:29:58 (1.69 MB/s) - ‘MNIST.tar.gz’ saved [34813078]

MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw/tra

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./
    Split: Train

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
mnist_trainset = torchvision.datasets.MNIST(root=root_dir, train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = torchvision.datasets.MNIST(root=root_dir, 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

In [5]:
# class_inds 이거는 그냥 위에있는거를 list 로 만들어준 형태임 
class_inds = [torch.where(mnist_trainset.targets == class_idx)[0]
              for class_idx in mnist_trainset.class_to_idx.values()]

In [6]:
train_dataloaders = [
                     DataLoader(dataset=Subset(mnist_trainset, inds),
                                batch_size=50,
                                shuffle=True,
                                drop_last=False
                     )
                     for inds in class_inds
]

test_dataloader  = torch.utils.data.DataLoader(mnist_testset, 
                                               batch_size=50, 
                                               shuffle=False)

print("Training dataset size: ", len(mnist_trainset))
print("Testing dataset size: ",  len(mnist_testset))

Training dataset size:  60000
Testing dataset size:  10000


In [7]:
# ************* modify this section for later use *************
# Define the model 
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # modify this section for later use 
        self.linear_1 = torch.nn.Linear(784, 768)
        self.linear_2 = torch.nn.Linear(768, 10)
        self.sigmoid12  = torch.nn.Sigmoid()

        self.layer_activations = dict()

    def forward(self, x):

        # modify this section for later use 
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid12(x)
        pred = self.linear_2(x)
        return pred
# ************* modify this section for later use *************

In [8]:
def get_activation(model, layer_name):    
    def hook(module, input, output):
        model.layer_activations[layer_name] = output
    return hook

In [9]:
def sparsity_calculator(final_spareness):
    sparseness_list = list()
    for single_epoch_spareness in final_spareness:

        hidden_layer_activation_list = single_epoch_spareness
        hidden_layer_activation_list = torch.stack(hidden_layer_activation_list)
        layer_activations_list = torch.reshape(hidden_layer_activation_list, (10000, 768))

        layer_activations_list = torch.abs(layer_activations_list)  # modified 
        num_neurons = layer_activations_list.shape[1]
        population_sparseness = (np.sqrt(num_neurons) - (torch.sum(layer_activations_list, dim=1) / torch.sqrt(torch.sum(layer_activations_list ** 2, dim=1)))) / (np.sqrt(num_neurons) - 1)
        mean_sparseness_per_epoch = torch.mean(population_sparseness)

        sparseness_list.append(mean_sparseness_per_epoch)

    return sparseness_list

In [10]:
def selectivity(hidden_layer_each_neuron):
    __selectivity__ = list()
    # I will now try to find the average of each class for each neuron.
    # check out the next cell 
    avg_activations = [dict() for x in range(768)]
    for i, neuron in enumerate(hidden_layer_each_neuron):
        for k, v in neuron.items():
            # v is the list of activations for hidden layer's neuron k 
            avg_activations[i][k] = sum(v) / float(len(v))

    # generate 768 lists to get only values in avg_activations
    only_activation_vals = [list() for x in range(768)]

    # get only values from avg_activations
    for i, avg_activation in enumerate(avg_activations):
        for value in avg_activation.values():
            only_activation_vals[i].append(value)


    for activation_val in only_activation_vals:
        # find u_max 
        u_max = np.max(activation_val)

        # find u_minus_max 
        u_minus_max = (np.sum(activation_val) - u_max) / 9

        # find selectivity 
        selectivity = (u_max - u_minus_max) / (u_max + u_minus_max)

        # append selectivity value to selectivity
        __selectivity__.append(selectivity)

    avg_selectivity = np.average(__selectivity__)
    std_selectivity = np.std(__selectivity__)
                                 
    return avg_selectivity, std_selectivity

In [11]:
# ************* modify this section for later use *************
def avg_std_calculator(_hidden_layer_each_neuron_12):

    avg_selectivity12, std_selectivity12 = selectivity(_hidden_layer_each_neuron_12)

    final_selectivity_avg = (avg_selectivity12) / 1
    final_selecvitity_std = (std_selectivity12) / 1

    return final_selectivity_avg, final_selecvitity_std
# ************* modify this section for later use *************

In [12]:
def model_factory(optimizer_name, seed_num):
    '''
    optimizer_name : choose one of Adagrad, Adadelta, SGD, and Adam 

    '''
    my_model = Model()
    print("my_model:", my_model)
    my_model.to(device)

    # ************* modify this section for later use *************
    my_model.sigmoid12.register_forward_hook(get_activation(my_model, 's12'))
    # ************* modify this section for later use *************

    if optimizer_name == 'Adadelta':
        my_optimizer = torch.optim.Adadelta(my_model.parameters(), lr=1.0)

    elif optimizer_name == 'Adagrad':
        my_optimizer = torch.optim.Adagrad(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'SGD':
        my_optimizer = torch.optim.SGD(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'Adam':
        my_optimizer = torch.optim.Adam(my_model.parameters(), lr=0.001)

    else:
        print("ERROR")
    
    print("my_optimizer:", my_optimizer)
    test_acc, sparsity_avg, selectivity_list_avg, selectivity_list_std = sparsity_selectivity_trainer(optimizer=my_optimizer, model=my_model)

    # ************* modify this section for later use *************
    file_saver = open(f"seed_{seed_num}_neurons768_single_{optimizer_name}.txt", "w")
    # ************* modify this section for later use *************
    file_saver.write(str(test_acc)+'\n'+str(sparsity_avg)+'\n'+str(selectivity_list_avg)+'\n'+str(selectivity_list_std)+'\n\n')
    file_saver.close()

    if seed_num == 1:
    # ************* modify this section for later use *************
        if optimizer_name == 'Adadelta':
            !cp seed_1_neurons768_single_Adadelta.txt /content/drive/MyDrive
        
        elif optimizer_name == 'Adagrad':
            !cp seed_1_neurons768_single_Adagrad.txt /content/drive/MyDrive

        elif optimizer_name == 'SGD':
            !cp seed_1_neurons768_single_SGD.txt /content/drive/MyDrive

        elif optimizer_name == 'Adam':
            !cp seed_1_neurons768_single_Adam.txt /content/drive/MyDrive
        # ************* modify this section for later use *************
    
    elif seed_num == 100:
        # ************* modify this section for later use *************
        if optimizer_name == 'Adadelta':
            !cp seed_100_neurons768_single_Adadelta.txt /content/drive/MyDrive
        
        elif optimizer_name == 'Adagrad':
            !cp seed_100_neurons768_single_Adagrad.txt /content/drive/MyDrive

        elif optimizer_name == 'SGD':
            !cp seed_100_neurons768_single_SGD.txt /content/drive/MyDrive

        elif optimizer_name == 'Adam':
            !cp seed_100_neurons768_single_Adam.txt /content/drive/MyDrive
        # ************* modify this section for later use *************

    elif seed_num == 1234:
        # ************* modify this section for later use *************
        if optimizer_name == 'Adadelta':
            !cp seed_1234_neurons768_single_Adadelta.txt /content/drive/MyDrive
        
        elif optimizer_name == 'Adagrad':
            !cp seed_1234_neurons768_single_Adagrad.txt /content/drive/MyDrive

        elif optimizer_name == 'SGD':
            !cp seed_1234_neurons768_single_SGD.txt /content/drive/MyDrive

        elif optimizer_name == 'Adam':
            !cp seed_1234_neurons768_single_Adam.txt /content/drive/MyDrive
        # ************* modify this section for later use *************

    else:
        print("ERROR")

In [13]:
def sparsity_selectivity_trainer(optimizer, model):
    criterion = torch.nn.CrossEntropyLoss()
    no_epochs = 30
    test_acc   = list()

    selectivity_avg_list = list()
    selectivity_std_list = list()

    # ************* modify this section for later use *************
    final_spareness_12 = list()
    # ************* modify this section for later use *************

    for epoch in range(no_epochs):

        print(f"epoch {epoch} started")
        # ************* modify this section for later use *************
        hidden_layer_each_neuron_12 = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(768)]
        hidden_layer_each_neuron_12 = np.array(hidden_layer_each_neuron_12)
        # ************* modify this section for later use *************

        # ************* modify this section for later use *************
        hidden_layer_activation_list_12 = list()
        # ************* modify this section for later use *************

        # TRAINING 
        model.train()
        iterators = list(map(iter, train_dataloaders))   
        while iterators:
            iterator = np.random.choice(iterators)
            try:
                images, labels = next(iterator)   
                images, labels = images.to(device), labels.to(device)

                optimizer.zero_grad()

                pred = model(images)

                loss = criterion(pred, labels)

                loss.backward()
                optimizer.step()
                
            except StopIteration:
                iterators.remove(iterator)

        # TESTING
        model.eval()
        total = 0
        for itr, (images, labels) in enumerate(test_dataloader):
            images, labels = images.to(device), labels.to(device)

            pred = model(images)

            loss = criterion(pred, labels)

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if labels[i] == torch.max(p.data, 0)[1]:
                    total = total + 1

            # ***************** sparsity calculation ***************** #
            hidden_layer_activation_list_12.append(model.layer_activations['s12'])

            # ************* modify this section for later use *************
            for activation, label in zip(model.layer_activations['s12'], labels):
                label = label.item()
                with torch.no_grad():
                    activation = activation.numpy()
                for i in range(768):    
                    hidden_layer_each_neuron_12[i][label].append(activation[i])
        
        selectivity_avg, selecvitity_std = avg_std_calculator(hidden_layer_each_neuron_12)
        # ************* modify this section for later use *************
            
        selectivity_avg_list.append(selectivity_avg)
        selectivity_std_list.append(selecvitity_std)

        # this conains activations for all epochs 
        final_spareness_12.append(hidden_layer_activation_list_12)
        # ***************** sparsity calculation ***************** # 

        accuracy = total / len(mnist_testset)

        # append accuracy here
        test_acc.append(accuracy)

        print('\nEpoch: {}/{}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, accuracy))
    
    print(selectivity_avg_list)
    print(selectivity_std_list)

    # ***************** sparsity calculation ***************** #
    sparsity_list12 = sparsity_calculator(final_spareness_12)

    average_sparsity = list()
    for i in range(no_epochs):
        average_sparsity.append( (sparsity_list12[i].item()) / 1 )
    # ***************** sparsity calculation ***************** #

    print("average_sparsity:", average_sparsity)

    return test_acc, average_sparsity, selectivity_avg_list, selectivity_std_list

# Seed 1

In [14]:
torch.manual_seed(1)
np.random.seed(1)

model_factory('Adam', 1)

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=768, bias=True)
  (linear_2): Linear(in_features=768, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)
epoch 0 started

Epoch: 1/30, Test Accuracy: 0.61700000
epoch 1 started

Epoch: 2/30, Test Accuracy: 0.83180000
epoch 2 started

Epoch: 3/30, Test Accuracy: 0.79750000
epoch 3 started

Epoch: 4/30, Test Accuracy: 0.89860000
epoch 4 started

Epoch: 5/30, Test Accuracy: 0.86810000
epoch 5 started

Epoch: 6/30, Test Accuracy: 0.91640000
epoch 6 started

Epoch: 7/30, Test Accuracy: 0.92150000
epoch 7 started

Epoch: 8/30, Test Accuracy: 0.92620000
epoch 8 started

Epoch: 9/30, Test Accuracy: 0.93410000
epoch 9 started

Epoch: 10/30, Test Accuracy: 0.95740000
epoch 10 started

Epoch: 11/30, Test Accuracy: 0.96820000
epoch 11 started

Epoch: 12/30, Test Accuracy: 0.95770000
epoch 12 star

# Seed 100

In [15]:
torch.manual_seed(100)
np.random.seed(100)

model_factory('Adam', 100)

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=768, bias=True)
  (linear_2): Linear(in_features=768, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)
epoch 0 started

Epoch: 1/30, Test Accuracy: 0.68330000
epoch 1 started

Epoch: 2/30, Test Accuracy: 0.72500000
epoch 2 started

Epoch: 3/30, Test Accuracy: 0.84710000
epoch 3 started

Epoch: 4/30, Test Accuracy: 0.88560000
epoch 4 started

Epoch: 5/30, Test Accuracy: 0.90820000
epoch 5 started

Epoch: 6/30, Test Accuracy: 0.88980000
epoch 6 started

Epoch: 7/30, Test Accuracy: 0.92370000
epoch 7 started

Epoch: 8/30, Test Accuracy: 0.94190000
epoch 8 started

Epoch: 9/30, Test Accuracy: 0.95790000
epoch 9 started

Epoch: 10/30, Test Accuracy: 0.95690000
epoch 10 started

Epoch: 11/30, Test Accuracy: 0.97010000
epoch 11 started

Epoch: 12/30, Test Accuracy: 0.96610000
epoch 12 star

# Seed 1234

In [16]:
torch.manual_seed(1234)
np.random.seed(1234)

model_factory('Adam', 1234)

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=768, bias=True)
  (linear_2): Linear(in_features=768, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)
epoch 0 started

Epoch: 1/30, Test Accuracy: 0.69780000
epoch 1 started

Epoch: 2/30, Test Accuracy: 0.75240000
epoch 2 started

Epoch: 3/30, Test Accuracy: 0.84310000
epoch 3 started

Epoch: 4/30, Test Accuracy: 0.88450000
epoch 4 started

Epoch: 5/30, Test Accuracy: 0.90080000
epoch 5 started

Epoch: 6/30, Test Accuracy: 0.92420000
epoch 6 started

Epoch: 7/30, Test Accuracy: 0.93870000
epoch 7 started

Epoch: 8/30, Test Accuracy: 0.93740000
epoch 8 started

Epoch: 9/30, Test Accuracy: 0.93170000
epoch 9 started

Epoch: 10/30, Test Accuracy: 0.95500000
epoch 10 started

Epoch: 11/30, Test Accuracy: 0.95200000
epoch 11 started

Epoch: 12/30, Test Accuracy: 0.96610000
epoch 12 star